In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
# 建构双隐层模型
# 输入层
x = tf.placeholder(tf.float32, [None, 784], name='X')

In [4]:
# 输入预览
image_shaped_input = tf.reshape(x, [-1, 28, 28, 1])
# 最多显示10张图片
tf.summary.image('input', image_shaped_input, 10)

<tf.Tensor 'input:0' shape=() dtype=string>

In [5]:
# 构建模型
# 此为两层NN示例
H1_NN = 256
H2_NN = 64

# 定义全连接层函数
def fcn_layer(inputs,           # 数入数据
              input_dim,        # 输入神经元数量
              output_dim,       # 输出神经元数量
              activation=None): # 激活函数
    
    # 以截断正态分布的随机函数初始化
    W = tf.Variable(tf.truncated_normal([input_dim, output_dim], stddev=0.1))
    # 以0初始化b
    b = tf.Variable(tf.zeros([output_dim]))
    
    # 建立权重计算
    XWb = tf.matmul(inputs, W) + b
    
    # 激活函数
    if activation is None:
        outputs = XWb
    else:
        outputs = activation(XWb)
        
    return outputs

In [6]:
# 隐藏层1
h1 = fcn_layer(x, 784, H1_NN, tf.nn.relu)
# 隐藏层2
h2 = fcn_layer(h1, H1_NN, H2_NN, tf.nn.relu)
# 输出层
forward = fcn_layer(h2, H2_NN, 10, None)
pred = tf.nn.softmax(forward)

In [7]:
# 直方图查看forward
tf.summary.histogram('forward', forward)

<tf.Tensor 'forward:0' shape=() dtype=string>

In [8]:
y = tf.placeholder(tf.float32, [None, 10], name='Y')

In [10]:
# 基础参数
train_epoch = 40
report_step = 1
learning_step = 0.01
batch_size = 50
total_batch = int(mnist.train.num_examples/batch_size)

In [11]:
# 定义损失函数
loss_func = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=forward, labels=y))

# 定义优化选择器
optimizer = tf.train.AdamOptimizer(learning_step).minimize(loss_func)

In [12]:
# TensorBoard
# 标量显示准确率
tf.summary.scalar('loss', loss_func)

<tf.Tensor 'loss:0' shape=() dtype=string>

In [13]:
# 定义准确率
correct_pred = tf.equal(tf.argmax(y, 1), tf.argmax(pred, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [14]:
# TensorBoard
# 标量显示准确率
tf.summary.scalar('accuracy', accuracy)

<tf.Tensor 'accuracy:0' shape=() dtype=string>

In [15]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [16]:
# 合并所有summary
merged_summary_op = tf.summary.merge_all()
# 创建写入符
writer = tf.summary.FileWriter('log/', sess.graph)

In [17]:
# 训练模型
from time import time
startTime = time()

for epoch in range(train_epoch):
    for batch in range(total_batch):
        xs, ys = mnist.train.next_batch(batch_size)
        sess.run(optimizer, feed_dict={x: xs, y: ys})
        
        # 生成summary
        summary_str = sess.run(merged_summary_op, feed_dict={x: xs, y: ys})
        writer.add_summary(summary_str, epoch) # 写入文件
    
    loss, acc = sess.run([loss_func, accuracy], feed_dict={x: mnist.validation.images, y: mnist.validation.labels})
    
    if (epoch + 1) % report_step == 0:
        print('Epoch: %02d' % (epoch+1), 'Loss: %.9f' % (loss), 'Accuracy: %.4f' % (acc))

    
# 显示运行时间
duration = time() - startTime
print('Finished. Train takes %.2f duration' % (duration))  

Epoch: 01 Loss: 0.126681954 Accuracy: 0.9638
Epoch: 02 Loss: 0.154044524 Accuracy: 0.9632
Epoch: 03 Loss: 0.176267192 Accuracy: 0.9636
Epoch: 04 Loss: 0.152023867 Accuracy: 0.9656
Epoch: 05 Loss: 0.144861177 Accuracy: 0.9712
Epoch: 06 Loss: 0.153188571 Accuracy: 0.9678
Epoch: 07 Loss: 0.161168963 Accuracy: 0.9682
Epoch: 08 Loss: 0.147775933 Accuracy: 0.9706
Epoch: 09 Loss: 0.206385374 Accuracy: 0.9640
Epoch: 10 Loss: 0.147989243 Accuracy: 0.9698
Epoch: 11 Loss: 0.146855205 Accuracy: 0.9744
Epoch: 12 Loss: 0.159313276 Accuracy: 0.9726
Epoch: 13 Loss: 0.194928467 Accuracy: 0.9670
Epoch: 14 Loss: 0.157807782 Accuracy: 0.9754
Epoch: 15 Loss: 0.185740471 Accuracy: 0.9698
Epoch: 16 Loss: 0.173885673 Accuracy: 0.9752
Epoch: 17 Loss: 0.199122638 Accuracy: 0.9704
Epoch: 18 Loss: 0.163601518 Accuracy: 0.9718
Epoch: 19 Loss: 0.202405378 Accuracy: 0.9716
Epoch: 20 Loss: 0.172507256 Accuracy: 0.9720
Epoch: 21 Loss: 0.205950633 Accuracy: 0.9722
Epoch: 22 Loss: 0.244425923 Accuracy: 0.9744
Epoch: 23 

In [18]:
sess.close()